## Task 3 - Hypothesis Testing

Repeat Task 3.2 after rescaling all features in the range [10, 15]. Use hypothesis testing to compare the performance of the models developed in this task with that developed in Tasks 3.2.

Markdown Question: What was the hypothesis you rejected in this task? How does p-value affect your decision?

<span style="font-weight:bold;">Save your response (program and markdown) as task3.ipynb</span>

In [ ]:
hypotheses testing for classification